In [ ]:
!pip install tensorflow_hub

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

import requests
from PIL import Image
from io import BytesIO

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Wczytywanie obrazu
image_url = "https://github.com/jgrynczewski/tensorflow_intro/raw/master/tiger.jpg"

response = requests.get(image_url)
# zrzutowanie odpowiedzi do obiektu Image biblioteki PIL
color_img = Image.open(BytesIO(response.content))

# zrzutowanie obiektu Image do tablicy numpy
color_data = np.asarray(color_img)

plt.imshow(color_data)

In [ ]:
# Wczytanie wyuczonego klasyfikatora
# https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_s/classification/2

efficientnetv2s_model_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_s/classification/2"
classifier = hub.load(efficientnetv2s_model_url)

In [ ]:
# Wczytanie etykiet
labels_file = "https://raw.githubusercontent.com/jgrynczewski/tensorflow_intro/master/ImageNetLabels.txt"


downloaded_file = tf.keras.utils.get_file("labels.txt", origin=labels_file)

classes = []

with open(downloaded_file) as f:
  labels = f.readlines()
  classes = [l.strip() for l in labels]

print(classes)

In [ ]:
# Preprocessing
# Zmiana rozmiarów i rzutowanie obrazu na tf.Tensor + zmiana rozmiarów
# Na podstawie https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_s/classification/2

image_size = 384

# zmiana kształtu na [batch_size, height, width, num_channels]
img = tf.reshape(color_data, [1, color_data.shape[0], color_data.shape[1], color_data.shape[2]])

# metoda `convert_image_dtype` do konwersji floats do zakresu [0,1]
image = tf.image.convert_image_dtype(img, tf.float32)

img_reshaped = tf.image.resize_with_pad(image, image_size, image_size)

print(image)

In [ ]:
# Przekazanie obrazu do modelu
probabilities = tf.nn.softmax(classifier(image)).numpy()
print(probabilities)
print(probabilities.shape)

top_5 = tf.argsort(probabilities, axis=-1, direction="DESCENDING")[0][:5].numpy()
np_classes = np.array(classes)

# Niektóre modele przewidują dodatkowo 'background', należy to uwzględnić 
# wczytując etykiety.
includes_background_class = probabilities.shape[1] == 1001

# 5 najlepszych prognoz
for i, item in enumerate(top_5):
  class_index = item if includes_background_class else item + 1
  line = f'({i+1}) {class_index:4} - {classes[class_index]}: {probabilities[0][top_5][i]}'
  print(line)

# # 
# image_size = image.shape[1]
# plt.figure(figsize=(image_size, image_size))
plt.imshow(image[0])
plt.show()


Transfer wiedzy (*ang. Transfer learning*)